# Implementation of Simulated Annealing

## Initial Model
We will start with 0-Rule model, which is a simple baseline classification model. It does not use any features for prediction but instead predicts the most frequent class (mode) in the training dataset. This can serve as a baseline to compare the performance of more sophisticated models.

In [6]:
import numpy as np
import time
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.dummy import DummyClassifier


class AutoML:
    def __init__(self, initial_temp=100, cooling_rate=0.99, max_iterations=100, min_training_time=3600):
        self.initial_temp = initial_temp
        self.cooling_rate = cooling_rate
        self.max_iterations = max_iterations
        self.min_training_time = min_training_time

        self.algorithms = {
            'DecisionTreeClassifier': {
                'class': DecisionTreeClassifier,
                'parameters': ['max_depth', 'min_samples_split'],
                'ranges': [(1, 20), (2, 20)]
            },
            'SVC': {
                'class': SVC,
                'parameters': ['C', 'gamma'],
                'ranges': [(0.01, 10), (0.01, 1)]
            },
            'RandomForestClassifier': {
                'class': RandomForestClassifier,
                'parameters': ['n_estimators', 'max_depth'],
                'ranges': [(10, 100), (1, 20)]
            },
            'GradientBoostingClassifier': {
                'class': GradientBoostingClassifier,
                'parameters': ['n_estimators', 'learning_rate'],
                'ranges': [(10, 100), (0.01, 0.3)]
            },
            'MLPClassifier': {
                'class': MLPClassifier,
                'parameters': ['alpha', 'learning_rate_init'],
                'ranges': [(0.0001, 0.1), (0.0001, 0.1)]
            },
            'GaussianNB': { 
                'class': GaussianNB,
                'parameters': [],  
                'ranges': []      
            }
        }

        self.best_solution = None
        self.best_score = 0
        self.model = None

    def eval(self, model, X, y):
        scores = cross_val_score(model, X, y, cv=5)
        print('scores=' + str(np.mean(scores)))
        return np.mean(scores)

    def generate_neighborhood(self, current_solution):
        if not isinstance(current_solution, list):
            current_solution = [current_solution.__class__.__name__]
    
        algorithm_name = np.random.choice(list(self.algorithms.keys()))
        algorithm_info = self.algorithms[algorithm_name]
    
        new_solution = [algorithm_name] + [None] * len(algorithm_info['parameters'])
    
        for i, parameter in enumerate(algorithm_info['parameters']):
            if len(algorithm_info['ranges']) == 0:
                continue 
    
            low, high = algorithm_info['ranges'][i]
    
            if low is not None and high is not None:
                if isinstance(high, list):  
                    new_solution[i + 1] = np.random.choice(high)
                elif isinstance(high, str):  
                    current_idx = algorithm_info['ranges'][i].index(high)
                    new_idx = (current_idx - 1) % len(algorithm_info['ranges'][i])
                    new_solution[i + 1] = algorithm_info['ranges'][i][new_idx]
                elif isinstance(low, int) and isinstance(high, int): 
                    new_solution[i + 1] = np.random.randint(low, high)
                else:  
                    new_solution[i + 1] = np.random.uniform(low, high)
    
        print(f"Neighborhood algorithm: {algorithm_name}, parameters: {new_solution[1:]}")
        return new_solution

    def create_model(self, solution):
        algorithm_name = solution[0]
        hyperparameters = solution[1:]
        algorithm_class = self.algorithms[algorithm_name]['class']
        if algorithm_name == 'MLPClassifier':
            return algorithm_class(alpha=hyperparameters[0], learning_rate_init=hyperparameters[1])
        elif algorithm_name == 'GaussianNB':
            return algorithm_class()
        else:
            return algorithm_class(**{param: int(value) if param in ['max_depth', 'n_estimators', 'min_samples_split'] else value for param, value in zip(self.algorithms[algorithm_name]['parameters'], hyperparameters)})

    def fit(self, X, y):
        self.X = X
        self.y = y
        self.simulated_annealing()

    def predict(self, X):
        if self.model is None:
            raise ValueError("The model has not been fit yet. Please call the fit method first.")
        return self.model.predict(X)

    def simulated_annealing(self):
        start_time = time.time()  
        # 0 rule model as initial model as base model
        zero_r_model = DummyClassifier(strategy='most_frequent')
        zero_r_model.fit(self.X, self.y) 

        print(f"Initial model: DummyClassifier")
        print(f"Initial parameters: (strategy='most_frequent')")

        current_solution = ['DummyClassifier']
        current_score = self.eval(zero_r_model, self.X, self.y)
        best_solution = current_solution
        best_score = current_score
    
        temperature = self.initial_temp
    
        while time.time() - start_time < self.min_training_time:
            for _ in range(10):
                new_solution = self.generate_neighborhood(current_solution)
                new_score = self.eval(self.create_model(new_solution), self.X, self.y)
    
                if new_score > current_score:
                    current_solution = new_solution
                    current_score = new_score
                    if new_score > best_score:
                        best_solution = new_solution
                        best_score = new_score
                else:
                    acceptance_probability = np.exp((new_score - current_score) / temperature)
                    if np.random.rand() < acceptance_probability:
                        current_solution = new_solution
                        current_score = new_score
    
            temperature *= self.cooling_rate
    
        self.best_solution = best_solution
        self.best_score = best_score
        self.model = self.create_model(best_solution)
        self.model.fit(self.X, self.y)
        print(f'best_score is {best_score}')
        print(f'best_solution is {best_solution}')


X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, random_state=42)
automl = AutoML(min_training_time=10) 
automl.fit(X, y)
predictions = automl.predict(X)


Initial model: DummyClassifier
Initial parameters: (strategy='most_frequent')
scores=0.503
Neighborhood algorithm: GradientBoostingClassifier, parameters: [79, 0.07230287258803343]
scores=0.9100000000000001
Neighborhood algorithm: GradientBoostingClassifier, parameters: [49, 0.11697764672954684]
scores=0.909
Neighborhood algorithm: GaussianNB, parameters: []
scores=0.8470000000000001
Neighborhood algorithm: DecisionTreeClassifier, parameters: [6, 10]
scores=0.893
Neighborhood algorithm: DecisionTreeClassifier, parameters: [1, 19]
scores=0.719
Neighborhood algorithm: DecisionTreeClassifier, parameters: [10, 10]
scores=0.9
Neighborhood algorithm: GradientBoostingClassifier, parameters: [37, 0.2538584183023678]
scores=0.9259999999999999
Neighborhood algorithm: DecisionTreeClassifier, parameters: [3, 5]
scores=0.873
Neighborhood algorithm: GradientBoostingClassifier, parameters: [10, 0.2856864970748658]
scores=0.905
Neighborhood algorithm: DecisionTreeClassifier, parameters: [3, 9]
scores=